In [ ]:
# !source ~/.secrets.sh


In [ ]:
%load_ext watermark


In [ ]:
import boto3
import botocore
from io import StringIO
from iterdub import iterdub as ib
from iterpop import iterpop as ip
from matplotlib import pyplot as plt
import pandas as pd
from pandas.util import hash_pandas_object
import re
import seaborn as sns
import statsmodels.api as sm
from teeplot import teeplot as tp


In [ ]:
import os

from IPython.display import display
import seaborn as sns
from teeplot import teeplot as tp

from dishpylib.pyhelpers import make_outattr_metadata
from dishpylib.pyhelpers import print_runtime


In [ ]:
%watermark -diwmuv -iv


# get data


In [ ]:
s3_handle = boto3.resource(
    's3',
    region_name="us-east-2",
    config=botocore.config.Config(
        signature_version=botocore.UNSIGNED,
    ),
)
bucket_handle = s3_handle.Bucket('prq49')

series_profiles, = bucket_handle.objects.filter(
    Prefix=f'endeavor=16/series-profiles/stage=8+what=elaborated/',
)


In [ ]:
df = pd.read_csv(
    f's3://prq49/{series_profiles.key}',
    compression='xz',
)
dfdigest = '{:x}'.format(hash_pandas_object( df ).sum())
dfdigest


# set up graphing utilities


In [ ]:
def letterscatter(*,data,x,y,cat):
    sns.scatterplot(
        data=data,
        x=x,
        y=y,
        color='k',
        edgecolor='k',
        s=200,
    )
    sns.scatterplot(
        data=data,
        x=x,
        y=y,
        hue=cat,
        edgecolor=None,
        s=120,
    )
    sns.scatterplot(
        data=data,
        x=x,
        y=y,
        color='w',
        style=cat,
        markers={
            phen : f'$\mathrm{{{phen}}}$'
            for phen in data[cat].unique()
        },
        legend=False,
    )
    plt.legend(
        bbox_to_anchor=(1.01, 1),
        borderaxespad=0,
        title='Morph',
        frameon=False,
    )


In [ ]:
def letterscatter_vline(*args, **kwargs):
    letterscatter(*args, **kwargs)
    plt.axvline(15,zorder=-2,c='k',ls=':')
    plt.axvline(45,zorder=-2,c='k',ls='--')


In [ ]:
def letterscatter_vline_interpolationmissing(*args, **kwargs):
    letterscatter_vline(*args, **kwargs)

    for stint in dfx[ dfx['a=most_credible_idx+set_size=1'].isnull() ]['Stint']:
        plt.axvline(stint, c='gray', zorder=-2, alpha=0.5)


# preprocess data


In [ ]:
morph_csv = StringIO('''Stint,Morph,Series
0,a,16005
1,b,16005
2,c,16005
3,b,16005
4,b,16005
5,b,16005
6,b,16005
7,b,16005
8,b,16005
9,b,16005
10,b,16005
11,b,16005
12,b,16005
13,b,16005
14,d,16005
15,e,16005
16,e,16005
17,e,16005
18,e,16005
19,e,16005
20,e,16005
21,e,16005
22,e,16005
23,e,16005
24,e,16005
25,e,16005
26,b,16005
27,e,16005
28,b,16005
29,e,16005
30,e,16005
31,e,16005
32,e,16005
33,e,16005
34,e,16005
35,e,16005
36,e,16005
37,e,16005
38,e,16005
39,f,16005
40,e,16005
41,e,16005
42,e,16005
43,e,16005
44,e,16005
45,g,16005
46,g,16005
47,g,16005
48,g,16005
49,g,16005
50,g,16005
51,g,16005
52,g,16005
53,e,16005
54,g,16005
55,g,16005
56,g,16005
57,g,16005
58,g,16005
59,h,16005
60,g,16005
61,e,16005
62,g,16005
63,g,16005
64,e,16005
65,e,16005
66,g,16005
67,g,16005
68,e,16005
69,g,16005
70,e,16005
71,e,16005
72,e,16005
73,g,16005
74,i,16005
75,i,16005
76,g,16005
77,i,16005
78,e,16005
79,g,16005
80,e,16005
81,e,16005
82,e,16005
83,e,16005
84,e,16005
85,e,16005
86,e,16005
87,e,16005
88,e,16005
89,g,16005
90,g,16005
91,e,16005
92,i,16005
93,e,16005
94,e,16005
95,b,16005
96,b,16005
97,h,16005
98,e,16005
99,e,16005
100,j,16005
''')

df_morph = pd.read_csv(morph_csv)


In [ ]:
df = pd.merge(
    df,
    df_morph,
    on=['Stint', 'Series'],
    how='outer',
)


In [ ]:
lowestroot_phylo_csv = StringIO('''proc,thread,Stint,Series,Lowest Num Stint Root IDs,Lowest Root ID
0,0,0,16005,1,2378
0,1,0,16005,1,2378
0,2,0,16005,1,2378
0,3,0,16005,1,2378
0,0,100,16005,25,12634
0,1,100,16005,26,12634
0,2,100,16005,24,12634
0,3,100,16005,22,12634
0,0,10,16005,4,12634
0,1,10,16005,3,12634
0,2,10,16005,4,12634
0,3,10,16005,3,12634
0,0,11,16005,3,12634
0,1,11,16005,3,12634
0,2,11,16005,2,12634
0,3,11,16005,3,12634
0,0,12,16005,4,12634
0,1,12,16005,4,12634
0,2,12,16005,4,12634
0,3,12,16005,4,12634
0,0,13,16005,2,12634
0,1,13,16005,2,12634
0,2,13,16005,2,12634
0,3,13,16005,2,12634
0,0,14,16005,3,12634
0,1,14,16005,3,12634
0,2,14,16005,3,12634
0,3,14,16005,3,12634
0,0,15,16005,3,12634
0,1,15,16005,4,12634
0,2,15,16005,4,12634
0,3,15,16005,4,12634
0,0,16,16005,4,12634
0,1,16,16005,3,12634
0,2,16,16005,3,12634
0,3,16,16005,2,12634
0,0,17,16005,9,12634
0,1,17,16005,5,12634
0,2,17,16005,5,12634
0,3,17,16005,3,12634
0,0,18,16005,7,12634
0,1,18,16005,7,12634
0,2,18,16005,3,12634
0,3,18,16005,6,12634
0,0,19,16005,8,12634
0,1,19,16005,11,12634
0,2,19,16005,6,12634
0,3,19,16005,7,12634
0,0,1,16005,1,2378
0,1,1,16005,1,2378
0,2,1,16005,1,2378
0,3,1,16005,2,2378
0,0,20,16005,8,12634
0,1,20,16005,9,12634
0,2,20,16005,9,12634
0,3,20,16005,11,12634
0,0,21,16005,10,12634
0,1,21,16005,9,12634
0,2,21,16005,11,12634
0,3,21,16005,6,12634
0,0,22,16005,11,12634
0,1,22,16005,10,12634
0,2,22,16005,8,12634
0,3,22,16005,10,12634
0,0,23,16005,7,12634
0,1,23,16005,10,12634
0,2,23,16005,15,12634
0,3,23,16005,13,12634
0,0,24,16005,10,12634
0,1,24,16005,5,12634
0,2,24,16005,13,12634
0,3,24,16005,7,12634
0,0,25,16005,13,12634
0,1,25,16005,15,12634
0,2,25,16005,5,12634
0,3,25,16005,9,12634
0,0,26,16005,6,12634
0,1,26,16005,6,12634
0,2,26,16005,7,12634
0,3,26,16005,8,12634
0,0,27,16005,11,12634
0,1,27,16005,9,12634
0,2,27,16005,9,12634
0,3,27,16005,6,12634
0,0,28,16005,8,12634
0,1,28,16005,6,12634
0,2,28,16005,13,12634
0,3,28,16005,9,12634
0,0,29,16005,7,12634
0,1,29,16005,8,12634
0,2,29,16005,9,12634
0,3,29,16005,10,12634
0,0,2,16005,2,12634
0,1,2,16005,3,12634
0,2,2,16005,2,12634
0,3,2,16005,2,12634
0,0,30,16005,10,12634
0,1,30,16005,10,12634
0,2,30,16005,16,12634
0,3,30,16005,11,12634
0,0,31,16005,11,12634
0,1,31,16005,12,12634
0,2,31,16005,9,12634
0,3,31,16005,12,12634
0,0,32,16005,10,12634
0,1,32,16005,9,12634
0,2,32,16005,14,12634
0,3,32,16005,15,12634
0,0,33,16005,12,12634
0,1,33,16005,18,12634
0,2,33,16005,13,12634
0,3,33,16005,19,12634
0,0,34,16005,23,12634
0,1,34,16005,25,12634
0,2,34,16005,17,12634
0,3,34,16005,22,12634
0,0,35,16005,13,12634
0,1,35,16005,15,12634
0,2,35,16005,14,12634
0,3,35,16005,13,12634
0,0,36,16005,13,12634
0,1,36,16005,17,12634
0,2,36,16005,16,12634
0,3,36,16005,23,12634
0,0,37,16005,8,12634
0,1,37,16005,12,12634
0,2,37,16005,9,12634
0,3,37,16005,17,12634
0,0,38,16005,16,12634
0,1,38,16005,16,12634
0,2,38,16005,18,12634
0,3,38,16005,15,12634
0,0,39,16005,19,12634
0,1,39,16005,21,12634
0,2,39,16005,10,12634
0,3,39,16005,14,12634
0,0,3,16005,6,12634
0,1,3,16005,6,12634
0,2,3,16005,4,12634
0,3,3,16005,7,12634
0,0,40,16005,16,12634
0,1,40,16005,15,12634
0,2,40,16005,17,12634
0,3,40,16005,17,12634
0,0,41,16005,18,12634
0,1,41,16005,19,12634
0,2,41,16005,26,12634
0,3,41,16005,20,12634
0,0,42,16005,18,12634
0,1,42,16005,17,12634
0,2,42,16005,16,12634
0,3,42,16005,12,12634
0,0,43,16005,22,12634
0,1,43,16005,26,12634
0,2,43,16005,18,12634
0,3,43,16005,18,12634
0,0,44,16005,20,12634
0,1,44,16005,14,12634
0,2,44,16005,17,12634
0,3,44,16005,13,12634
0,0,45,16005,8,12634
0,1,45,16005,17,12634
0,2,45,16005,14,12634
0,3,45,16005,18,12634
0,0,46,16005,14,12634
0,1,46,16005,16,12634
0,2,46,16005,14,12634
0,3,46,16005,16,12634
0,0,47,16005,11,12634
0,1,47,16005,11,12634
0,2,47,16005,16,12634
0,3,47,16005,25,12634
0,0,48,16005,12,12634
0,1,48,16005,17,12634
0,2,48,16005,15,12634
0,3,48,16005,15,12634
0,0,49,16005,13,12634
0,1,49,16005,11,12634
0,2,49,16005,13,12634
0,3,49,16005,15,12634
0,0,4,16005,5,12634
0,1,4,16005,4,12634
0,2,4,16005,3,12634
0,3,4,16005,5,12634
0,0,50,16005,15,12634
0,1,50,16005,13,12634
0,2,50,16005,12,12634
0,3,50,16005,11,12634
0,0,51,16005,13,12634
0,1,51,16005,14,12634
0,2,51,16005,9,12634
0,3,51,16005,11,12634
0,0,52,16005,24,12634
0,1,52,16005,14,12634
0,2,52,16005,15,12634
0,3,52,16005,21,12634
0,0,53,16005,17,12634
0,1,53,16005,13,12634
0,2,53,16005,16,12634
0,3,53,16005,21,12634
0,0,54,16005,12,12634
0,1,54,16005,12,12634
0,2,54,16005,13,12634
0,3,54,16005,14,12634
0,0,55,16005,14,12634
0,1,55,16005,11,12634
0,2,55,16005,12,12634
0,3,55,16005,17,12634
0,0,56,16005,12,12634
0,1,56,16005,19,12634
0,2,56,16005,13,12634
0,3,56,16005,15,12634
0,0,57,16005,24,12634
0,1,57,16005,23,12634
0,2,57,16005,20,12634
0,3,57,16005,19,12634
0,0,58,16005,17,12634
0,1,58,16005,17,12634
0,2,58,16005,17,12634
0,3,58,16005,16,12634
0,0,59,16005,18,12634
0,1,59,16005,13,12634
0,2,59,16005,13,12634
0,3,59,16005,12,12634
0,0,5,16005,5,12634
0,1,5,16005,6,12634
0,2,5,16005,6,12634
0,3,5,16005,6,12634
0,0,60,16005,20,12634
0,1,60,16005,19,12634
0,2,60,16005,18,12634
0,3,60,16005,20,12634
0,0,61,16005,14,12634
0,1,61,16005,16,12634
0,2,61,16005,9,12634
0,3,61,16005,14,12634
0,0,62,16005,12,12634
0,1,62,16005,11,12634
0,2,62,16005,10,12634
0,3,62,16005,15,12634
0,0,63,16005,12,12634
0,1,63,16005,13,12634
0,2,63,16005,13,12634
0,3,63,16005,15,12634
0,0,64,16005,14,12634
0,1,64,16005,13,12634
0,2,64,16005,11,12634
0,3,64,16005,10,12634
0,0,65,16005,12,12634
0,1,65,16005,15,12634
0,2,65,16005,13,12634
0,3,65,16005,13,12634
0,0,66,16005,16,12634
0,1,66,16005,15,12634
0,2,66,16005,17,12634
0,3,66,16005,13,12634
0,0,67,16005,10,12634
0,1,67,16005,8,12634
0,2,67,16005,6,12634
0,3,67,16005,11,12634
0,0,68,16005,8,12634
0,1,68,16005,10,12634
0,2,68,16005,12,12634
0,3,68,16005,10,12634
0,0,69,16005,9,12634
0,1,69,16005,11,12634
0,2,69,16005,9,12634
0,3,69,16005,9,12634
0,0,6,16005,4,12634
0,1,6,16005,3,12634
0,2,6,16005,4,12634
0,3,6,16005,4,12634
0,0,70,16005,13,12634
0,1,70,16005,16,12634
0,2,70,16005,13,12634
0,3,70,16005,16,12634
0,0,71,16005,17,12634
0,1,71,16005,18,12634
0,2,71,16005,18,12634
0,3,71,16005,18,12634
0,0,72,16005,16,12634
0,1,72,16005,15,12634
0,2,72,16005,16,12634
0,3,72,16005,18,12634
0,0,73,16005,17,12634
0,1,73,16005,19,12634
0,2,73,16005,15,12634
0,3,73,16005,21,12634
0,0,74,16005,17,12634
0,1,74,16005,21,12634
0,2,74,16005,17,12634
0,3,74,16005,22,12634
0,0,75,16005,13,12634
0,1,75,16005,12,12634
0,2,75,16005,15,12634
0,3,75,16005,14,12634
0,0,76,16005,14,12634
0,1,76,16005,15,12634
0,2,76,16005,16,12634
0,3,76,16005,20,12634
0,0,77,16005,9,12634
0,1,77,16005,9,12634
0,2,77,16005,13,12634
0,3,77,16005,12,12634
0,0,78,16005,14,12634
0,1,78,16005,12,12634
0,2,78,16005,13,12634
0,3,78,16005,12,12634
0,0,79,16005,16,12634
0,1,79,16005,21,12634
0,2,79,16005,22,12634
0,3,79,16005,21,12634
0,0,7,16005,4,12634
0,1,7,16005,4,12634
0,2,7,16005,5,12634
0,3,7,16005,5,12634
0,0,80,16005,14,12634
0,1,80,16005,12,12634
0,2,80,16005,11,12634
0,3,80,16005,12,12634
0,0,81,16005,13,12634
0,1,81,16005,17,12634
0,2,81,16005,21,12634
0,3,81,16005,17,12634
0,0,82,16005,16,12634
0,1,82,16005,17,12634
0,2,82,16005,18,12634
0,3,82,16005,13,12634
0,0,83,16005,12,12634
0,1,83,16005,16,12634
0,2,83,16005,12,12634
0,3,83,16005,11,12634
0,0,84,16005,19,12634
0,1,84,16005,17,12634
0,2,84,16005,12,12634
0,3,84,16005,14,12634
0,0,85,16005,11,12634
0,1,85,16005,17,12634
0,2,85,16005,18,12634
0,3,85,16005,19,12634
0,0,86,16005,16,12634
0,1,86,16005,22,12634
0,2,86,16005,14,12634
0,3,86,16005,18,12634
0,0,87,16005,17,12634
0,1,87,16005,16,12634
0,2,87,16005,15,12634
0,3,87,16005,15,12634
0,0,88,16005,14,12634
0,1,88,16005,12,12634
0,2,88,16005,15,12634
0,3,88,16005,10,12634
0,0,89,16005,15,12634
0,1,89,16005,14,12634
0,2,89,16005,14,12634
0,3,89,16005,16,12634
0,0,8,16005,3,12634
0,1,8,16005,3,12634
0,2,8,16005,4,12634
0,3,8,16005,4,12634
0,0,90,16005,19,12634
0,1,90,16005,17,12634
0,2,90,16005,11,12634
0,3,90,16005,8,12634
0,0,91,16005,16,12634
0,1,91,16005,17,12634
0,2,91,16005,15,12634
0,3,91,16005,18,12634
0,0,92,16005,10,12634
0,1,92,16005,12,12634
0,2,92,16005,17,12634
0,3,92,16005,19,12634
0,0,93,16005,7,12634
0,1,93,16005,5,12634
0,2,93,16005,11,12634
0,3,93,16005,4,12634
0,0,94,16005,17,12634
0,1,94,16005,12,12634
0,2,94,16005,20,12634
0,3,94,16005,18,12634
0,0,95,16005,12,12634
0,1,95,16005,12,12634
0,2,95,16005,16,12634
0,3,95,16005,19,12634
0,0,96,16005,14,12634
0,1,96,16005,17,12634
0,2,96,16005,16,12634
0,3,96,16005,11,12634
0,0,97,16005,10,12634
0,1,97,16005,10,12634
0,2,97,16005,15,12634
0,3,97,16005,11,12634
0,0,98,16005,17,12634
0,1,98,16005,17,12634
0,2,98,16005,21,12634
0,3,98,16005,16,12634
0,0,99,16005,19,12634
0,1,99,16005,15,12634
0,2,99,16005,21,12634
0,3,99,16005,15,12634
0,0,9,16005,6,12634
0,1,9,16005,5,12634
0,2,9,16005,3,12634
0,3,9,16005,5,12634
''')

df_lowestroot_phylo = pd.read_csv(lowestroot_phylo_csv).groupby([
    'Series',
    'Stint',
]).mean().reset_index()


In [ ]:
df = pd.merge(
    df,
    df_lowestroot_phylo,
    on=['Stint', 'Series'],
    how='outer',
)


In [ ]:
dfm10 = df[ df['Stint'] % 10 == 0 ]


In [ ]:
dfx = df[ df['Series'] == 16005 ].sort_values('Morph')


In [ ]:
for col in df.columns:
    print(col)


## Fitness Data


In [ ]:
background_key = {
    "immediatepredecessor-simuls-dosecorrected-withbioticbackground-withmut"
        : "Prefatory",
    "immediatepredecessor-simuls-dosecorrected-withfuturebioticbackground-withmut"
        : "Contemporary",
    "immediatepredecessor-simuls-nobioticbackground-withmut"
        : "Without",
    "lowestroot-immediatepredecessor-battles"
        : "Without",
    "immediatepredecessor-simuls-dosecorrected-nodiversitymaintenance-withbioticbackground-withmut"
        : "Prefatory\n(no diversity maint.)",
    "immediatepredecessor-simuls-dosecorrected-nodiversitymaintenance-withfuturebioticbackground-withmut"
        : "Contemporary\n(no diversity maint.)",
    "immediatepredecessor-battles-dosecorrected-withbioticbackground-withmut"
        : "Prefatory",
    "immediatepredecessor-battles-dosecorrected-withfuturebioticbackground-withmut"
        : "Contemporary",
}


In [ ]:
color_key = {
    'a' : 'tab:blue',
    'b' : 'tab:orange',
    'c' : 'tab:green',
    'd' : 'tab:red',
    'e' : 'tab:purple',
    'f' : 'tab:brown',
    'g' : 'tab:pink',
    'h' : 'tab:gray',
    'i' : 'tab:olive',
    'j' : 'tab:cyan',
}


In [ ]:
morph_key = {
    row['Stint'] : row['Morph']
    for idx, row in df_morph.iterrows()
}


In [ ]:
def subject_key(setup: str) -> str:
    if 'simul' in setup or 'competition' in setup:
        return 'Specimen'
    elif 'battle' in setup:
        return 'Population'
    else:
        assert False


In [ ]:
dfs_fit = []
for setup in (
    "immediatepredecessor-simuls-dosecorrected-withbioticbackground-withmut",
    "immediatepredecessor-simuls-dosecorrected-withfuturebioticbackground-withmut",
    "immediatepredecessor-simuls-nobioticbackground-withmut",
    "lowestroot-immediatepredecessor-battles",
    "immediatepredecessor-simuls-dosecorrected-nodiversitymaintenance-withbioticbackground-withmut",
    "immediatepredecessor-simuls-dosecorrected-nodiversitymaintenance-withfuturebioticbackground-withmut",
    "immediatepredecessor-battles-dosecorrected-withbioticbackground-withmut",
    "immediatepredecessor-battles-dosecorrected-withfuturebioticbackground-withmut",
):
    print(setup)
    df_fit = pd.read_csv(
        f"https://prq49.s3.us-east-2.amazonaws.com/endeavor%3D16/{setup}/stage%3D2%2Bwhat%3Dcollated/a%3Dconcat%2Bhow%3Dmanual%2Bext%3D.csv",
    )
    df_fit["setup"] = setup
    if "Fitness Differential Focal" not in df_fit:
        df_fit["Fitness Differential Focal"] = df_fit["Fitness Differential"]
    if "Focal Prevalence" not in df_fit:
        df_fit["Focal Prevalence"] = df_fit["Prevalence"]
    if "Focal Abundance" not in df_fit:
        df_fit["Focal Abundance"] = df_fit["Abundance"]

    df_fit['Positive Fitness Differential Focal'] = df_fit['Fitness Differential Focal'] > 0

    dfs_fit.append(df_fit)


In [ ]:
df_fit = pd.concat(dfs_fit, ignore_index=True)
df_fit['Biotic Background'] = df_fit['setup'].apply(lambda setup: background_key[setup])
df_fit['Assay Subject'] = df_fit['setup'].apply(lambda setup: subject_key(setup))
df_fit['Assay Subject'] = df_fit['setup'].apply(lambda setup: subject_key(setup))


In [ ]:
df_fit = df_fit[
    (df_fit["Root ID"] == 1)
    & (df_fit["Competition Stint"] > 0)
]


In [ ]:
# adapted from https://stackoverflow.com/a/49601444
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    r, g, b = mc.to_rgb(c)
    return r + amount * (1 - r), g + amount * (1 - g), b + amount * (1 - b)


In [ ]:
df["Mutational Sensitivity"] = 1 - df["Fraction Mutations that are Deleterious"]


In [ ]:
prev_stint = [  # from phylo
    0,  # 0
    1,  # 1
    2,  # 2
    2,  # 3
    3,  # 4
    3,  # 5
    3,  # 6
    6,  # 7
    6,  # 8
    5,  # 9
    9,  # 10
    10, # 11
    9,  # 12
    12,  # 13
    13,  # 14
    13,  # 15
    15,  # 16
    16,  # 17
    17,  # 18
    18,  # 19
    18,  # 20
    20,  # 21
    21,  # 22
    19,  # 23
    23,  # 24
    24,  # 25
    24,  # 26
    24,  # 27
    26,  # 28
    27,  # 29
    29,  # 30
    29,  # 31
    29,  # 32
    30,  # 33
    32,  # 34
    34,  # 35
    33,  # 36
    36,  # 37
    37,  # 38
    36,  # 39
    38,  # 40
    39,  # 41
    40,  # 42
    41,  # 43
    42,  # 44
    42,  # 45
    45,  # 46
    46,  # 47
    47,  # 48
    46,  # 49
    49,  # 50
    50,  # 51
    50,  # 52
    52,  # 53
    49,  # 54
    49,  # 55
    55,  # 56
    56,  # 57
    55,  # 58
    56,  # 59
    57,  # 60
    60,  # 61
    61,  # 62
    57,  # 63
    63,  # 64
    64,  # 65
    63,  # 66
    66,  # 67
    64,  # 68
    63,  # 69
    68,  # 70
    69,  # 71
    69,  # 72
    72,  # 73
    68,  # 74
    73,  # 75
    73,  # 76
    76,  # 77
    68,  # 78
    76,  # 79
    78,  # 80
    79,  # 81
    80,  # 82
    80,  # 83
    83,  # 84
    83,  # 85
    85,  # 86
    80,  # 87
    85,  # 88
    87,  # 89
    89,  # 90
    90,  # 91
    89,  # 92
    90,  # 93
    93,  # 94
    93,  # 95
    93,  # 96
    94,  # 97
    94,  # 98
    98,  # 99
    99, # 100
]
import numpy as np
prev_stint_np = np.array(prev_stint)

## Plotting


In [ ]:
import matplotlib.pyplot as plt

# Create 10 columns, making the last one 10% wider
widths = [1] * 9 + [1.1]  # first 9 columns equal, last column 10% wider

fig, (ax1, *axes) = plt.subplots(
    10, 10,
    sharex=False,
    figsize=(8.5, 4.5),
    gridspec_kw={"width_ratios": widths}
)

plt.subplots_adjust(hspace=0.0, wspace=0.2)


for i, ax in enumerate(axes[-1]):
    data = df_fit.sort_values(
        "Competition Stint",
    ).loc[
        (
            (df_fit["Competition Stint"].clip(lower=None, upper=99) // 10 == i)
            & (df_fit["Assay Subject"] == "Population")
            & (df_fit["Biotic Background"] == "Contemporary")
        ),
    ]
    data["Delta Prevalence"] = data["Focal Prevalence"] - 0.5
    data["Mean Delta Prevalence"] = data.groupby(
        "Competition Stint"
    )["Delta Prevalence"].transform("mean")
    sns.barplot(
        data=data,
        x="Competition Stint",
        y="Delta Prevalence",
        legend=False,
        palette="vlag_r",
        hue="Mean Delta Prevalence",
        hue_norm=plt.Normalize(
            vmin=-0.5,
            vmax=0.5,
        ),
        errorbar=None,
        dodge=False,
        ax=ax,
    )
    ax.set_ylim(
        -0.6,
        0.6,
    )
    ax.yaxis.label.set(rotation='horizontal', ha='right')
    ax.set_xticklabels([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.tick_params(axis="y", labelcolor="darkgray")
    if i < 9:
        ax.set_ylabel('')
        ax.set_yticks([])
    else:
        ax.yaxis.tick_right()
        ax.set_yticks([0])
        ax.set_yticklabels(["0.0"])

    ax.set_xlabel("Stint" if i == 4 else "")
    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.grid(
        True,
        axis="x",
        which="major",
        linestyle=":",
        lw=0.8,
        alpha=0.4,
    )
    ax.set_xticklabels([])
    if i == 0:
        ax.set_ylabel(
            "Δ Fit Population",
            rotation=30,
            ha='right',
            labelpad=5,
            color="#2d2018",
        )
    else:
        ax.set_ylabel('')

    if i < 9:
        ax.set_xticks([*range(10)])
        ax.set_xticklabels([str(i * 10), *list(" " * 9)])
    else:
        ax.set_xticks([*range(11)])
        ax.set_xticklabels([str(i * 10), *list(" " * 9), "100"])

for i, ax in enumerate(axes[-2]):
    data = df_fit.sort_values(
        "Competition Stint",
    ).loc[
        (
            (df_fit["Competition Stint"].clip(lower=None, upper=99) // 10 == i)
            & (df_fit["Assay Subject"] == "Specimen")
            & (df_fit["Biotic Background"] == "Contemporary")
        ),
    ]
    data["Delta Prevalence"] = data["Focal Prevalence"] - 0.5
    data["Mean Delta Prevalence"] = data.groupby(
        "Competition Stint"
    )["Delta Prevalence"].transform("mean")
    sns.barplot(
        data=data,
        x="Competition Stint",
        y="Delta Prevalence",
        legend=False,
        palette="vlag_r",
        hue="Mean Delta Prevalence",
        hue_norm=plt.Normalize(
            vmin=-0.5,
            vmax=0.5,
        ),
        errorbar=None,
        dodge=False,
        ax=ax,
    )
    ax.set_ylim(
        -0.6,
        0.6,
    )
    ax.yaxis.label.set(rotation='horizontal', ha='right')
    ax.set_xticklabels([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.tick_params(axis="y", labelcolor="darkgray")
    if i < 9:
        ax.set_ylabel('')
        ax.set_yticks([])
    else:
        ax.yaxis.tick_right()
        ax.set_yticks([0])
        ax.set_yticklabels(["0.0"])

    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.grid(
        True,
        axis="x",
        which="major",
        linestyle=":",
        lw=0.8,
        alpha=0.4,
    )
    ax.set_xticklabels([])
    if i == 0:
        ax.set_ylabel(
            "Δ Fit Specimen",
            rotation=30,
            ha='right',
            labelpad=5,
            color="#2d2018",
        )
    else:
        ax.set_ylabel('')

for i, ax in enumerate(ax1):
    data = df.sort_values(
        "Stint",
    ).loc[
        (
            (df["Series"] == 16005)
        ),
        ["Morph", "Stint"],
    ].replace(
        {
            "a": 0,
            "b": 1,
            "c": 2,
            "d": 3,
            "e": 4,
            "f": 5,
            "g": 6,
            "h": 7,
            "i": 8,
            "j": 9,
        },
    )
    data2 = data.copy()
    import numpy as np
    data2.loc[
        (
            data2["Morph"].duplicated()
            & (data2["Stint"] != 3)
        ) | data2["Stint"].isin([0, 1])
    , "Morph"] = np.nan

    data = data.loc[
        data["Stint"].clip(lower=None, upper=99) // 10 == i,
        ["Morph"],
    ]
    data2 = data2.loc[
        data2["Stint"].clip(lower=None, upper=99) // 10 == i,
        ["Morph"],
    ]

    sns.heatmap(
        data=data.to_numpy().T,
        cmap=[
            lighten_color(c, 0.5)
            for c in sns.color_palette("pastel")
        ],
        ax=ax,
        cbar=False,
        vmin=0,
        vmax=9,
    )
    sns.heatmap(
        data=data2.to_numpy().T,
        cmap=sns.color_palette("tab10"),
        ax=ax,
        linewidths=0.01,
        cbar=False,
        vmin=0,
        vmax=9,
    )
    ax.set_yticks([])
    ax.set_xticklabels([])
    if i:
        ax.set_ylabel('')
        ax.set_yticks([])
    else:
        ax.set_ylabel(
            "Morph",
            rotation=30, ha='right', labelpad=5, color="darkslategray"
        )
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.set_xticks([])

from matplotlib.patches import ConnectionPatch
for fr, to in enumerate(prev_stint):
    if fr != to:
        assert fr > to
        ifr = fr // 10
        ito = to // 10
        xfr = fr % 10
        xto = to % 10
        if fr == 100:
            xfr = 10
            ifr = 9
        con = ConnectionPatch(
            xyA=(xfr + 0.5, 0), coordsA=ax1[ifr].transData,
            xyB=(xto + 0.5, 0), coordsB=ax1[ito].transData,
            arrowstyle="<-", #shrinkB=5,
            connectionstyle=f"arc3,rad={0.7 - (fr - to) * 0.02}",
            mutation_scale=5,
            zorder=-1,
        )
        fig.add_artist(con)

ys = [
    'Num Instructions',
    # "Mutational Sensitivity",
    'Critical Fitness Complexity',
    'Cardinal Interface Complexity',
    #'Resource Receiving Cell Fraction (evolve mean)',
    # 'Fraction Mutations that are Deleterious',
    #'Number Unique Module Expression Profiles',
    # 'Number Unique Module Regulation Profiles',
    #'Mean Program Module Count (monoculture mean)',
    'Fraction Deaths apoptosis (monoculture mean)',
    'Birth Conflict Ratio for Kin Commonality At Least 1',
    'Mean Kin Group Size Level 0 (monoculture first)',
    'Mean Kin Group Size Level 1 (monoculture first)',
]

for axrow, y in zip(axes, ys):
    vmin = df.sort_values(
        "Stint",
    ).loc[
        (
            (df["Series"] == 16005)
            & (df["Stint"] > 1)  # exclude non-related stints
        ),
        y,
    ].diff().min()
    vmax = df.sort_values(
        "Stint",
    ).loc[
        (
            (df["Series"] == 16005)
            & (df["Stint"] > 1)  # exclude non-related stints
        ),
        y,
    ].diff().max()

    from matplotlib import colors

    vcenter = 0
    norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

    for i, ax in enumerate(axrow):
        data = df.sort_values(
            "Stint",
        ).loc[
            (
                (df["Series"] == 16005)
                & (df["Stint"].clip(lower=None, upper=99) // 10 == i)
            ),
        ].copy()
        data["delta"] = data[y] - data[y].to_numpy()[
            prev_stint_np[np.arange(len(data))]
        ]
        sns.barplot(
            data=data,
            x="Stint",
            y=y,
            hue="delta",
            dodge=False,
            legend=False,
            ax=ax,
            palette="vlag_r",
            hue_norm=norm,
        )
        data["Stint"] -= data["Stint"].min()
        sns.lineplot(
            data=data,
            x="Stint",
            y=y,
            color=[
                "darkgoldenrod", "darkkhaki", "darkseagreen", "tan", "darksalmon", "rosybrown",
            ][
                ys.index(y) % 6
            ],
            lw=0.7,
            ax=ax,
            zorder=-10,
        )
        ax.set_ylim(
            0,
            df.loc[
                (df["Series"] == 16005),
                y,
            ].max() * 1.1,
        )
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        if i:
            ax.set_ylabel('')
        else:
            ax.set_ylabel(
                {
                    'Critical Fitness Complexity': "G Complexity",
                    'Cardinal Interface Complexity': "P Complexity",
                    'Fraction Deaths apoptosis (monoculture mean)': "Apoptosis",
                    'Birth Conflict Ratio for Kin Commonality At Least 1': "Kin Conflict",
                    'Fraction Mutations that are Deleterious': "Robustness",
                    'Num Instructions': "Genome Size",
                    'Mean Kin Group Size Level 0 (monoculture first)'
                        : 'Inner Group Size',
                    'Mean Kin Group Size Level 1 (monoculture first)'
                        : 'Outer Group Size',
                }[y],
                rotation=30,
                ha='right',
                labelpad=5,
                color={
                    'Critical Fitness Complexity': "black",
                    'Cardinal Interface Complexity': "black",
                    'Fraction Deaths apoptosis (monoculture mean)': "darkslategray",
                    'Birth Conflict Ratio for Kin Commonality At Least 1': "darkslategray",
                    'Fraction Mutations that are Deleterious': "darkslategray",
                    'Num Instructions': "black",
                    'Mean Kin Group Size Level 0 (monoculture first)'
                        : "darkslategray",
                    "Mean Kin Group Size Level 1 (monoculture first)"
                        : "darkslategray",
                }[y],
            )
        if y == "Birth Conflict Ratio for Kin Commonality At Least 1":
            # ax.set_yscale("symlog")
            ax.set_ylim(0, 1)
            ax.set_yticks([])


        from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
        ax.xaxis.set_major_locator(MultipleLocator(2))
        ax.grid(
            True,
            axis="x",
            which="major",
            linestyle=":",
            lw=0.8,
            alpha=0.4,
        )
        ax.set_xticklabels([])
        ax.yaxis.set_label_position("left")
        ax.tick_params(axis='y', labelcolor='darkgray')

        if i < 9:
            ax.set_yticks([])
        else:
            ax.yaxis.tick_right()
            ax.tick_params(axis="y", labelrotation=-30)
            if y == "Birth Conflict Ratio for Kin Commonality At Least 1":
                ax.set_yticks([1.0])
                ax.set_yticklabels(["1.0"])
            else:
                ax.set_yticks(
                    [df.loc[df["Series"] == 16005, y].max()]
                )

for j, axrow in enumerate(axes):
    for i, ax in enumerate(axrow):
        for x in [
            [2, 3],
            [4, 5],
            [],
            [9],
            [5],
            [9],
            [],
            [4],
            [],
            [3, 10],
        ][i]:
            ax.axvline(
                x=x,
                color='black',
                linestyle='--',
                zorder=-10,
                lw=0.8,
            )
        if i < 9:
            ax.set_yticks([])
        if i < 9 and j != len(axes) - 1:
            if i != 0:
                ax.set_axis_off()
            if i == 0:
                for side in ("top", "right", "bottom", "left"):
                    ax.spines[side].set_visible(False)

                # hide ticks & tick labels
                ax.tick_params(
                    left=False, bottom=False, labelleft=False, labelbottom=False
                )

    for i, ax in enumerate(axrow):
        for x in range(1, 11, 2):
            ax.axvline(
                x=x,
                color='darkgray',
                linestyle=':',
                zorder=-10,
                lw=0.8,
                alpha=0.9,
            )

        ax.set_xlim([-0.5, 9.5 + (i == 9)])

        if i != 5:
            ax.set_xlabel("")

        if i < 9:
            if j == len(axes) - 1:
                ax.set_xticks([*range(10)])
                ax.set_xticklabels([str(i * 10), *list(" " * 9)])
            else:
                ax.set_xticks([*range(11)])
                ax.set_xticklabels([str(i * 10), *list(" " * 9), "100"])


for i, x, y, m in (
    (0, 2.5, 0.7, 2),
    (0, 3.5, 1.1, 1),
    (1, 4.5, 0.7, 3),
    (1, 5.5, 1.1, 4),
    (3, 9.5, 0.7, 5),
    (4, 5.5, 0.7, 6),
    (5, 9.5, 0.7, 7),
    (7, 4.5, 0.7, 8),
    (9, 10.5, 0.7, 9),
):
    fontsize = 8

    from matplotlib.transforms import blended_transform_factory
    # Use data coords for x and axes-fraction for y
    ax = ax1[i]
    trans = blended_transform_factory(ax.transData, ax.transAxes)
    y_top = 1.0
    y_circle = y_top + y

    # Vertical line descending to the top of the axis
    ax.plot([x, x], [y_circle, y_top],
            transform=trans,
            color='black',
            linestyle='--',
            zorder=-10,
            lw=0.8,
            clip_on=False)

    # Filled circle + white bold 'c' label
    tab10_3rd = plt.get_cmap('tab10')(m)
    ax.text(x, y_circle, "abcdefghijk"[m],
            transform=trans,
            ha='center', va='center',
            fontsize=fontsize, fontweight='bold',
            color='white', zorder=100, clip_on=False,
            bbox=dict(boxstyle='circle,pad=0.15',
                    facecolor=tab10_3rd, edgecolor='none'))


from matplotlib import pyplot as plt
plt.subplots_adjust(hspace=0.0, wspace=0.2)
plt.show()
